Based on the code shown in https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_sift.html

In [237]:
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.feature import SIFT, match_descriptors, plot_matched_features
from PIL import Image
import numpy as np
import os
import cv2
from skimage.measure import ransac
from skimage.transform import ProjectiveTransform
from pathlib import Path
import pandas as pd
from joblib import Parallel, delayed

In [238]:
# Load the datafiles and make them grayscale
def load_image(path):
    return rgb2gray(np.array(Image.open(path)))

In [239]:
# Used to acces the name of the folder we are working on : not necessary
print("Répertoire courant :", os.getcwd())


Répertoire courant : c:\Users\coren\OneDrive\Bureau\corentin\Etudes\polytech\cours\semestre9\DMC4100\Assignment5\Augmented_reality_system\src


In [240]:
# Extract SIFT keypoints and descriptors
def extract_sift_features(image):
    sift = SIFT()
    sift.detect_and_extract(image)
    return sift.keypoints, sift.descriptors

Problem of dimensions for the first two pictures.

In [241]:
def resize_to_match(img1, img2):
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    return cv2.resize(img2, (w1, h1)) if (h1 != h2 or w1 != w2) else img2

In [242]:
# Visualize the matches
def match_and_plot(img1, img2, keypoints1, descriptors1, keypoints2, descriptors2, title):
    matches = match_descriptors(descriptors1, descriptors2, max_ratio=0.6, cross_check=True)

    #img2 = resize_to_match(img1, img2)
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    plt.gray()
    plot_matched_features(image0=img1, image1=img2, 
                          keypoints0=keypoints1, keypoints1=keypoints2,
                          matches=matches, ax=ax, 
                          keypoints_color='cyan',
                          only_matches=True)
    ax.axis('off')
    ax.set_title(title)
    plt.show()
    return matches

In [243]:
# Compute matching accuracy using optional homography
def compute_matching_accuracy(keypoints1, keypoints2, matches, homography=None, threshold=5):
    """
    Compute the matching accuracy based on a homography.
    If no homography is provided, only the number of matches is printed.
    """
    if homography is None:
        print(f"{len(matches)} matches found (no validation)")
        return None

    if matches is None or len(matches) == 0:
        print("No matches to evaluate.")
        return 0

    correct = 0
    for i, j in matches:
        pt1 = np.array([*keypoints1[i], 1.0])  # homogeneous coordinates
        projected = homography @ pt1
        projected /= projected[2]  # normalize

        pt2 = keypoints2[j]
        error = np.linalg.norm(projected[:2] - pt2)

        if error < threshold:
            correct += 1

    accuracy = correct / len(matches) if matches.size > 0 else 0
    print(f"Matching Accuracy: {accuracy:.2%} ({correct}/{len(matches)} correct matches)")
    return accuracy

In [244]:
def extract_patch(img, keypoint, size=21):
    x, y = int(keypoint[0]), int(keypoint[1])
    half = size // 2
    patch = img[max(0, y - half):y + half + 1, max(0, x - half):x + half + 1]
    if patch.shape != (size, size):
        return None  # Ignore les bords
    return patch


In [245]:
def normalized_cross_correlation(patch1, patch2):
    """
    Calcule la corrélation croisée normalisée entre deux patchs d'image.
    
    Args:
        patch1 (np.ndarray): Premier patch (grayscale, 2D array).
        patch2 (np.ndarray): Deuxième patch (même taille que patch1).
    
    Returns:
        float: Score de corrélation croisée normalisée (entre -1 et 1).
    """
    if patch1.shape != patch2.shape:
        raise ValueError("Les deux patchs doivent avoir la même taille.")

    # Centrage des patchs
    patch1_mean = patch1 - np.mean(patch1)
    patch2_mean = patch2 - np.mean(patch2)

    # Calcul du score NCC
    numerator = np.sum(patch1_mean * patch2_mean)
    denominator = np.sqrt(np.sum(patch1_mean**2) * np.sum(patch2_mean**2))

    if denominator == 0:
        return 0.0  # Cas dégénéré : patch uniforme

    return numerator / denominator


In [246]:

def match_by_ncc_parallel(img_ref, img_test, keypoints_ref, keypoints_test, threshold=0.7):
    def match_one(i, kp_test):
        patch_test = extract_patch(img_test, kp_test)
        if patch_test is None:
            return None

        best_score = -1
        best_idx = -1
        for j, kp_ref in enumerate(keypoints_ref):
            patch_ref = extract_patch(img_ref, kp_ref)
            if patch_ref is None:
                continue
            score = normalized_cross_correlation(patch_test, patch_ref)
            if score > best_score:
                best_score = score
                best_idx = j

        if best_score >= threshold:
            return cv2.DMatch(_queryIdx=i, _trainIdx=best_idx, _distance=1 - best_score)
        return None

    results = Parallel(n_jobs=-1)(delayed(match_one)(i, kp) for i, kp in enumerate(keypoints_test))
    return [m for m in results if m is not None]


In [247]:
def estimate_homography_from_matches(keypoints_ref, keypoints_test, matches, ransac_thresh=5.0):
    """
    Estime l'homographie entre deux images à partir des correspondances de points clés.

    Args:
        keypoints_ref (list of cv2.KeyPoint): Points clés de l'image de référence.
        keypoints_test (list of cv2.KeyPoint): Points clés de l'image test.
        matches (list of cv2.DMatch): Correspondances entre les points.
        ransac_thresh (float): Seuil RANSAC pour l'estimation robuste.

    Returns:
        H (np.ndarray): Matrice d'homographie 3x3 ou None si estimation échoue.
    """
    if len(matches) < 4:
        return None  # Pas assez de points pour estimer une homographie

    pts_ref = np.float32([keypoints_ref[m.trainIdx] for m in matches])
    pts_test = np.float32([keypoints_test[m.queryIdx] for m in matches])

    H, mask = cv2.findHomography(pts_ref, pts_test, cv2.RANSAC, ransac_thresh)
    return H


In [248]:
def build_dataset_index(dataset_dir):
    """
    Analyse toutes les images du dataset et stocke les informations clés dans un DataFrame.

    Args:
        dataset_dir (str or Path): Dossier contenant les images de référence.

    Returns:
        pd.DataFrame: DataFrame contenant les informations extraites pour chaque image.
    """
    dataset_paths = sorted(Path(dataset_dir).rglob("*"))
    records = []

    for ref_path in dataset_paths:
        if ref_path.suffix.lower() not in [".jpg", ".jpeg", ".png", ".avif"]:
            continue

        img_ref = load_image(str(ref_path))
        keypoints_ref, descriptors_ref = extract_sift_features(img_ref)

        if len(keypoints_ref) == 0:
            print(f"⚠️ Skipped {ref_path.name} due to missing keypoints")
            continue

        record = {
            "filename": ref_path.name,
            "path": str(ref_path),
            "num_keypoints": len(keypoints_ref),
            "keypoints": keypoints_ref,
            "descriptors": descriptors_ref,
            "image_shape": img_ref.shape,
            "image": img_ref
        }
        records.append(record)

    df = pd.DataFrame(records)
    return df




In [249]:


def batch_compare_tests_to_dataset(test_dir, dataset_dir, ncc_threshold=0.7):
    dataset_df = build_dataset_index(dataset_dir)
    record ={}
    
    for test_path in sorted(Path(test_dir).glob("*")):
        print(f"\n🧪 Test image: {test_path.name}")
        img_test = load_image(str(test_path))
        keypoints_test, _ = extract_sift_features(img_test)

        for _, row in dataset_df.iterrows():
            keypoints_ref = row["keypoints"]
            img_ref = row["image"]


            if len(keypoints_ref) == 0 or len(keypoints_test) == 0:
                print(f"⚠️ Skipped {row['filename']} due to missing keypoints")
                continue

            matches = match_by_ncc_parallel(img_ref, img_test, keypoints_ref, keypoints_test, threshold=ncc_threshold)
            if not matches:
                print(f"❌ No NCC matches between {row['filename']} and {test_path.name}")
                continue

            H = estimate_homography_from_matches(keypoints_ref, keypoints_test, matches)
            accuracy = compute_matching_accuracy(keypoints_ref, keypoints_test, matches, H)

            print(f"🔗 Accuracy of {row['filename']} vs {test_path.name}: {accuracy:.2f}")


In [250]:
batch_compare_tests_to_dataset("../data/test", "../data/images")


🧪 Test image: city_hall_test.jpg


ModuleNotFoundError: No module named '_posixsubprocess'